In [1]:
ESSAY = """The Power of Curiosity: Fueling Human Progress

Curiosity is one of the most powerful driving forces behind human innovation and discovery. It is the innate desire to explore, learn, and understand the world around us. From the earliest days of human history, curiosity has propelled societies forward, leading to remarkable advancements in science, technology, and culture. It is often said that curiosity killed the cat, but in reality, curiosity has been the very thing that has allowed humans to evolve and adapt in an ever-changing world.

"""

# Setup and Basic Text Generation

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Import necessary modules
import asyncio
from dotenv import load_dotenv
import os
from shuscribe.services.llm.session import LLMSession
from shuscribe.services.llm.providers.provider import (
    Message, GenerationConfig
)
from shuscribe.schemas.llm import MessageRole

load_dotenv()
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
ANTHROPIC_API_KEY = os.environ["ANTHROPIC_API_KEY"]
GEMINI_API_KEY = os.environ["GEMINI_API_KEY"]

TEST_MODELS ={
    "openai": "gpt-4o-mini",
    "anthropic": "claude-3-5-haiku-20241022",
    "gemini": "gemini-2.0-flash-001"
}

TEST_THINKING_MODELS = {
    "openai": "o3-mini-2025-01-31",
    "anthropic": "claude-3-7-sonnet-20250219",
    "gemini": "gemini-2.0-flash-thinking-exp"
}

# Helper function to run async code in notebook
async def run_async(coro):
    return await coro

## Basic Text Generation with Different Providers

In [4]:
# Set Provider Name
PROVIDER_NAME = "gemini"

In [9]:
async def test_gen():
    async with LLMSession.session_scope() as session:
        provider = await session.get_provider(PROVIDER_NAME)
        response = await provider.generate(
            messages=[
                Message(role=MessageRole.SYSTEM, content="You are a helpful assistant that speaks in a concise manner."),
                "What is the capital of France?"
                ],
            model=TEST_MODELS[PROVIDER_NAME],
            config=GenerationConfig(temperature=0.7)
        )
        return response.text

response = await run_async(test_gen())
print(f"{TEST_MODELS[PROVIDER_NAME]}:\n{response}")

gemini-2.0-flash-001:
Paris.



In [10]:
# Streaming response
async def test_streaming():
    async with LLMSession.session_scope() as session:
        # Create a streaming config
        config = GenerationConfig(temperature=0.7)
        
        print(f"{TEST_MODELS[PROVIDER_NAME]}:")

        async for chunk in session.generate_stream(
            messages=[
                Message(role=MessageRole.ASSISTANT, content=f"Sure! Here is the essay:\n{ESSAY}"),
                "Can you repeat exactly this essay that you generated?",
                Message(role=MessageRole.ASSISTANT, content="Sure!")]
            ,
            provider_name=PROVIDER_NAME,
            model=TEST_MODELS[PROVIDER_NAME],
            config=config
        ):
            print(chunk.text, end="", flush=True)

streaming_response = await run_async(test_streaming())


gemini-2.0-flash-001:
 Here is the essay:
The Power of Curiosity: Fueling Human Progress

Curiosity is one of the most powerful driving forces behind human innovation and discovery. It is the innate desire to explore, learn, and understand the world around us. From the earliest days of human history, curiosity has propelled societies forward, leading to remarkable advancements in science, technology, and culture. It is often said that curiosity killed the cat, but in reality, curiosity has been the very thing that has allowed humans to evolve and adapt in an ever-changing world.


In [11]:
# Using OpenAI with structured output
from pydantic import BaseModel, Field

class QAResponse(BaseModel):
    reasoning: str = Field(description="reasoning about the answer")
    response: str = Field(description="concise answer to the question")

async def test_openai_structured():
    async with LLMSession.session_scope() as session:
        print("Streaming response:")
        async for chunk in session.generate_stream(
            messages=[
                Message(role=MessageRole.SYSTEM, content="You are a helpful assistant"),
                "Does 1+1=3?"
            ],
            provider_name=PROVIDER_NAME,
            model=TEST_MODELS[PROVIDER_NAME], # or any model that supports response_format
            config=GenerationConfig(
                temperature=0.7,
                response_schema=QAResponse
            )
        ):
            print(chunk.text, end="", flush=True)

streaming_response = await run_async(test_openai_structured())

Streaming response:
{
  "reasoning": "Basic arithmetic dictates that 1 + 1 = 2, not 3.",
  "response": "No"
}

In [12]:
# thinking
from shuscribe.schemas.llm import ThinkingConfig

async def test_openai_structured():
    async with LLMSession.session_scope() as session:
        print("Streaming response:")
        async for chunk in session.generate_stream(
            messages=[
                Message(role=MessageRole.SYSTEM, content="You are a helpful assistant"),
                "Prove that 1+1=3"
            ],
            provider_name=PROVIDER_NAME,
            model=TEST_THINKING_MODELS[PROVIDER_NAME], # or any model that supports response_format
            config=GenerationConfig(
                temperature=0.7, # NOTE: TEMPERATURE IS NOT AVAILABLE FOR THINKING MODELS
                # response_schema=QAResponse, # NOTE: Gemini does not support response_schema for thinking models
                thinking_config=ThinkingConfig(enabled=True, effort="low")
            )
        ):
            print(chunk.text, end="", flush=True)

streaming_response = await run_async(test_openai_structured())


Streaming response:
Ah, are you ready for some mathematical trickery? 😉  While in standard mathematics, 1 + 1 definitely equals 2, we can have some fun and create a "proof" that it *looks* like 3, but it will rely on a hidden fallacy (a mistake in reasoning).

Here's a classic example of a false proof that tries to show 1+1=3:

**The "Proof" (which is WRONG):**

Let's assume:

1.  **a = b**
2.  **Multiply both sides by 'a':**  a² = ab
3.  **Subtract b² from both sides:** a² - b² = ab - b²
4.  **Factor both sides:** (a - b)(a + b) = b(a - b)
5.  **Divide both sides by (a - b):** a + b = b
6.  **Since we started with a = b, substitute 'a' for 'b':** a + a = a
7.  **Simplify:** 2a = a
8.  **Divide both sides by 'a':** 2 = 1
9.  **Add 1 to both sides:** 2 + 1 = 1 + 1
10. **Simplify:** 3 = 2

**Therefore, according to this "proof", 3 = 2, and if we manipulate it a bit more, we can get 1+1=3.**  For example, since 3=2, and we know 2 = 1+1, then substituting, we get 3 = 1+1, or 1+1=3.

**The 

# Session Management and Provider Reuse

In [13]:
# Test session management
async def test_session_reuse():
    results = []
    
    # Get the singleton instance
    session = await LLMSession.get_instance()
    
    # Use the same provider instance for multiple requests
    provider = await session.get_provider(PROVIDER_NAME)
    
    for i in range(3):
        response = await provider.generate(
            messages=[
                Message(role=MessageRole.USER, content=f"Count to {i+1} briefly.")
            ],
            model=TEST_MODELS[PROVIDER_NAME],
        )
        results.append(response.text)
    
    return results

session_results = await run_async(test_session_reuse())
for i, result in enumerate(session_results):
    print(f"Query {i+1}: {result}")

Query 1: 1

Query 2: 1, 2

Query 3: 1... 2... 3!

